In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Önceden eğitilmiş modeli yükle
model_path = "/content/drive/MyDrive/Colab Notebooks/cilt_kanseri_tespit_modelih5120.h5"  # Buraya model dosyanın yolunu yaz
model = load_model(model_path)


In [ ]:
# Yeni veri seti için klasör yolları
new_train_dir = "/content/drive/MyDrive/dataset/cnn_gelistirme/train"
new_test_dir = "/content/drive/MyDrive/dataset/cnnrs"


In [ ]:
# Veri artırma ve ön işleme
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   horizontal_flip=True,
                                   brightness_range=[0.8, 1.2],
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Yeni veri setini yükleme
train_generator = train_datagen.flow_from_directory(
    new_train_dir,
    target_size=(180, 180),  # Görsellerin boyutu 180x180 olacak şekilde güncellendi
    batch_size=32,
    class_mode='binary',  # İkili sınıflandırma için
    classes=['cancer_rs', 'non_cancer_rs']  # Ana sınıfları belirtiyoruz
)

test_generator = test_datagen.flow_from_directory(
    new_test_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    classes=['cancer_rs', 'non_cancer_rs']
)

Found 33127 images belonging to 2 classes.
Found 2041 images belonging to 2 classes.


In [ ]:
# Modelin son katmanlarını güncelle (opsiyonel)
for layer in model.layers[:-4]:  # İlk katmanları dondur, son 4 katmanı eğit
    layer.trainable = False


In [ ]:

# Modeli tekrar eğitme
# CENNET ESKİİ KOD ALTTAKİNİ BEN EKLEDİM HATADAN DOLAYI
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
#
## Assuming the last layer is Dense
#model.layers[-1].activation = tf.keras.activations.sigmoid  # Change the activation of the last layer
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
 #             loss='spars_categorical_crossentropy', # Keep the loss function as binary_crossentropy
  #            metrics=['accuracy'])


In [ ]:
model.fit(train_generator,
          validation_data=test_generator,
          epochs=80)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 15443s 15s/step - accuracy: 0.7337 - loss: 8.2450 - val_accuracy: 0.7967 - val_loss: 6.4317
Epoch 2/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 400s 386ms/step - accuracy: 0.9411 - loss: 1.1179 - val_accuracy: 0.7976 - val_loss: 5.3370
Epoch 3/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 401s 387ms/step - accuracy: 0.9409 - loss: 0.6834 - val_accuracy: 0.7991 - val_loss: 4.2519
Epoch 4/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 443s 387ms/step - accuracy: 0.9335 - loss: 0.5115 - val_accuracy: 0.7957 - val_loss: 3.4864
Epoch 5/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 397s 383ms/step - accuracy: 0.9210 - loss: 0.4688 - val_accuracy: 0.7972 - val_loss: 2.8997
Epoch 6/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 397s 384ms/step - accuracy: 0.9199 - loss: 0.3886 - val_accuracy: 0.7986 - val_loss: 2.5956
Epoch 7/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 394s 380ms/step - accuracy: 0.9265 - loss: 0.2994 - val_accuracy: 0.7986 - val_loss: 2.3689
Epoch 8/80
1036/1036 ━━━━━━━━━━━━━━━━━━━━ 395s 381ms/step - ac

In [ ]:
# Yeni modeli kaydetme
model.save("fine_tuned_model.h5")

In [ ]:
# Eğitim sonuçlarını görselleştirme
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epochs')
plt.ylabel('Doğruluk')
plt.legend()
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.show()

plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epochs')
plt.ylabel('Kayıp')
plt.legend()
plt.title('Eğitim ve Doğrulama Kaybı')
plt.show()



In [ ]:
import cv2
import numpy as np

def tahmin_yap(image_path):
    try:
        # Görüntüyü yükle
        image = cv2.imread(image_path)
        if image is None:
            return f"Görüntü yüklenemedi: {image_path}"

        # Görüntüyü yeniden boyutlandır ve normalleştir
        image = cv2.resize(image, (180, 180))  # Modelin boyutu
        image = image / 255.0  # Normalizasyon

        # Modelin giriş şekline uydur
        prediction = model(np.array([image]), training=False)  # Batch eklemeden çalıştır
        class_index = np.argmax(prediction)  # Sınıf tahmini
        class_name = categories[class_index]  # Sınıf ismi

        return class_name
    except Exception as e:
        return f"Hata oluştu: {e}"



# Örnek tahmin
sonuc = tahmin_yap("/content/drive/MyDrive/dataset/cnn_gelistirme/train/cancer_rs/ISIC_0232101.jpg")
print(f"Tahmin edilen sınıf: {sonuc}")


In [ ]:
# Test verileri üzerinde tahminler yap
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Sınıflandırma raporu
print("Sınıflandırma Raporu:")
print(classification_report(test_labels, predicted_labels, target_names=categories))

# Ek metrikler
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels, average='weighted')
recall = recall_score(test_labels, predicted_labels, average='weighted')
f1 = f1_score(test_labels, predicted_labels, average='weighted')

print(f"Doğruluk (Accuracy): {accuracy}")
print(f"Duyarlılık (Precision): {precision}")
print(f"Hatırlama (Recall): {recall}")
print(f"F1 Skoru: {f1}")
